In [1]:
# -*- coding: utf-8 -*- Line 2
#----------------------------------------------------------------------------
# Project     : Price Alarm System Enhancement Draft
# Created By  : Eungi Cho
# Created Date: 26/05/22
# Updated Date: 03/06/22
# version ='1.0'
# ---------------------------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import csv

warnings.filterwarnings("ignore")
plt.style.use('default')

In [2]:
import pathlib
print(pathlib.Path().absolute())
df_raw = pd.read_csv('/Users/cho-eungi/Practice/CSV/market_entry_price.csv')
print(df_raw.shape)
# print(df_raw.isnull().sum())
df_raw = df_raw.drop_duplicates()
df_raw.head()

/Users/cho-eungi/Practice/Tridge
(10619563, 11)


,source_id,country,market_id,product_id,entry_id,currency,final_unit,date,price_min,price_max,price_avg
0,201,South Africa,1487,131,92926374,ZAR,kg,2020-07-20,19.64,21.2,19.956
1,39,India,810,490,41039702,INR,kg,2020-07-06,11.80,12.5,12.200
2,41,India,2188,133,50157058,INR,kg,2020-07-06,50.00,52.7,51.400
3,556,Bangladesh,6581,545,84458922,BDT,kg,2020-07-13,4400.00,4800.0,4600.000
4,150,Turkey,2482,126,58387432,TRY,NaN,2020-07-13,10.00,15.0,11.288


In [12]:
# Create Test df
entry_lst = np.sort(df_raw['entry_id'].unique())
np.random.seed(0)
sample_entry = np.random.choice(entry_lst, 1000)
test_df = df_raw.loc[df_raw['entry_id'].isin(sample_entry)].sort_values(
    by = ['source_id', 'market_id', 'entry_id', 'date']).copy()
test_df['date'] = pd.to_datetime(test_df['date'])
test_df

,source_id,country,market_id,product_id,entry_id,currency,final_unit,date,price_min,price_max,price_avg
9270545,1,Netherlands,2775,99,41581966,USD,kg,2020-04-20,15.96,15.96,15.960
9045476,1,Netherlands,2775,99,41581966,USD,kg,2020-06-15,8.96,8.96,8.960
9235582,1,Netherlands,2775,99,41581966,USD,kg,2020-07-20,8.16,8.16,8.160
3920876,1,Netherlands,2782,113,41000912,USD,kg,2020-04-27,21.96,21.96,21.960
3821445,1,Netherlands,2782,113,41000912,USD,kg,2020-05-04,18.75,22.22,20.485
...,...,...,...,...,...,...,...,...,...,...,...
8855661,730,Vietnam,8784,15269,119373567,VND,kg,2022-03-28,22000.00,22000.00,22000.000
8855655,730,Vietnam,8784,15269,119373567,VND,kg,2022-04-04,20500.00,20500.00,20500.000
8855647,730,Vietnam,8784,15269,119373567,VND,kg,2022-04-11,20500.00,20500.00,20500.000
8855659,730,Vietnam,8784,15269,119373567,VND,kg,2022-04-18,19750.00,19750.00,19750.000


In [13]:
# W-MON date range from 2020 to 2022
# Left Join Test DF and Time DF
empty_df = pd.DataFrame()
for entry in sample_entry:
    entry_start = min(df_raw.loc[df_raw['entry_id'] == entry]['date'])
    date_range = pd.date_range(entry_start, '2022-05-31', freq = 'W-MON')
    time_df = pd.DataFrame({'date': date_range})
    
    time_df['entry_id_'] = entry
    entry_df = test_df.loc[test_df['entry_id'] == entry]
    joined_df = pd.merge(time_df, entry_df, left_on = ['date'], right_on = ['date'], how = 'left')
    empty_df = empty_df.append(joined_df)

df = empty_df.copy()
df = df.sort_values(by = ['entry_id_', 'date'])
df.set_index(np.arange(len(df)), inplace=True)
len(df.entry_id_.unique())

998

In [14]:
# Abnormal Price Range Detection
def exceed_3sigma(array):
    threshold_min = np.min(array) - 3 * np.std(array)
    threshold_max = np.min(array) + 3 * np.std(array)
    if threshold_min < 0:
        threshold_min = 0

    check_col = []
    for i in array:
        if i > threshold_max or i < threshold_min:
            check_col.append(1)
        else:
            check_col.append(0)
    return np.asarray(check_col)

# Count the occurances of consecutive null value
def count_consec_nan(array):
    consec_cnt = array.isnull().astype(int).groupby(array.notnull().astype(int).cumsum()).cumsum()
    return np.asarray(consec_cnt)

one_freq_0_lst1 = []
one_freq_0_lst2 = []
one_freq_not0_lst3 = []
multi_freq_lst1 = []
multi_freq_lst2 = []
multi_freq_lst3 = []
multi_freq_lst4 = []

# -- Alarm -- #
# price change rate
def alarm_1(df):
    df['price_avg_chg'] = np.where(
        (df['price_avg'].notnull()) & (df['price_avg'].shift(1).notnull())
        , df['price_avg'] / df['price_avg'].shift(1)
        , 0)
    df['rank'] = df.groupby('entry_id_')['date'].rank("dense", ascending = True)
    df['price_avg_chg_'] = np.where(df['rank'] == 1, np.nan, df['price_avg_chg'])
    df.drop(['price_avg_chg', 'rank'], axis = 1, inplace = True)
    df['alarm1'] = np.where(df['price_avg_chg_'] > 2, 1, 0)

# consecutive null count
def alarm_2(df):
    # 연속적으로 null 이 보고되는 횟수를 담은 column: consec_null
    # consec_null 의 shift(1) column: consec_null_shift
    df['consec_null'] = df.groupby('entry_id_')['price_avg'].transform(count_consec_nan)
    df['consec_null_shift'] = df.groupby('entry_id_')['consec_null'].shift(1)
    
    # Logic 상세 설명 - consec_null 와 consec_null_shift 비교:
    # entry_id의 Price의 첫 보고일 2020.10.05: consec_null == 0, consec_null_shift == NaN, freq == 0
    # 다음 Price 보고일 2020.10.12: consec_null == 0, consec_null_shift == 0, freq == 1
    # 다음 Price 보고일 2020.10.19: consec_null == 0, consec_null_shift == 0, freq == 1
    # 다음 Price 보고일 2020.11.09(3주차에 보고): consec_null == 0, consec_null_shift == 2, freq == 3
    
    # conditions[0]: Price 첫 보고일 // conditions[1]: 다음 Price 보고일
    # freq 결과값: 첫 보고일 = 0, 다음 보고일 = price 가 보고되지 않고 있던 주기 + 1, 보고 x = price 가 보고되지 않고 있던 주기
    conditions  = [ (df['consec_null'] == 0) & (df['consec_null_shift'].isnull()),
                   (df['consec_null'] == 0) & (df['consec_null_shift'].notnull())]
    choices     = [ 0, df['consec_null_shift'] + 1]
    df["freq"] = np.select(conditions, choices, default=np.nan)
    
    # 해당 entry_id의 Price의 주 보고 기간의 최빈값 계산. 복수 개의 최빈값이 있을 경우, 리스트 형태로 닮김.
    entry_freq = df.groupby('entry_id_')['freq'].agg(pd.Series.mode).to_dict()
    
    # 최빈값 Dict 생성
    threshold_dict = {}
    for entry, freq in entry_freq.items():
        # 최빈값이 단일값만 존재할 경우:
        if isinstance(freq, float):
            # 최빈값이 단일값만 존재하는 경우
            # freq = 0이라는 것은 단 한 번 Price가 보고된 후, 그 이후로는 보고되지 않고 있다는 뜻.
            if freq == 0:
                timedelt = round((pd.Timestamp(2022, 5, 31) - min(df.loc[df['entry_id_'] == entry]['date'])) / np.timedelta64(1, 'M'))
                # 2022/05/31 기준 2개월 이내 추가된 entry일 경우는 threshold를 100으로 설정.
#                 print('freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt = ', entry, freq, timedelt)
                if timedelt <= 2:
                    one_freq_0_lst1.append(entry)
                    threshold_dict[entry] = 100
                # 그렇지 않다면, threshold를 0 으로 설정 (즉, 이 경우에는 무조건 alarm 2에 포함)
                else:
                    one_freq_0_lst2.append(entry)
                    threshold_dict[entry] = 4
            else:
#                 print('freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq', entry, freq)
                one_freq_not0_lst3.append(entry)
#                 threshold_dict[entry] = 100
                threshold_dict[entry] = freq * 2
        # 최빈값이 두 개 이상 존재: 매우 불규칙하게 보고되어서 일정한 주기가 없는 경우
        # 이 때 threshold 설정은 0과 1을 제외한 min값
        else:
            # 최빈값이 [0, 1] 일 경우. 즉 최초 한 번과 바로 그 다음 주기에 price가 보고된 후 한 번도 들어오지 않은 경우
            if (np.min(freq) == 0) & (freq[np.where(freq == np.min(freq))[0][0] + 1] == 1) & (len(freq) == 2) :
                timedelt = round((pd.Timestamp(2022, 5, 31) - min(df.loc[df['entry_id_'] == entry]['date'])) / np.timedelta64(1, 'M'))
#                 print('freq 최빈값이 [0, 1]일 경우: entry, freq =', entry, freq)
                # 2022/05/31 기준 2개월 이내 추가된 entry일 경우는 threshold를 100으로 설정.
                if timedelt <= 2:
                    multi_freq_lst1.append(entry)
                    threshold_dict[entry] = 100
                # 그렇지 않다면, threshold를 0 으로 설정 (즉, 이 경우에는 무조건 alarm 2에 포함)
                else:
                    multi_freq_lst2.append(entry)
                    threshold_dict[entry] = 0
            # 최빈값이 [0,1,3,10,...] 등 최초 한 번 보고된 후에도 매우 불규칙적으로 보고되고 있는 entry일 경우
            else:
            # 위 예시같은 경우 - threshold = 3 (0의 index + 2) x 2
                value = freq[np.where(freq == np.min(freq))[0][0] + 1]
#                 print('freq 최빈값이 [0, 1] 이외에도 있는 경우: entry, freq, 0 next', entry, freq, value)
                if value == 1:
                    multi_freq_lst3.append(entry)
                    value = freq[np.where(freq == np.min(freq))[0][0] + 2]
#                     print('freq 최빈값이 [0, 1, 3, 10...]일 경우, 0 next', value)
                    threshold_dict[entry] = value * 2
                else:
                    multi_freq_lst4.append(entry)
#                     print('freq 최빈값이 [0, 3, 10...]일 경우, 0 next', value)
                    threshold_dict[entry] = value * 2
    def categorise(row):
        threshold_col = [threshold for entry, threshold in threshold_dict.items() if row['entry_id_'] == entry]
        return np.asarray(threshold_col[0])

    df['threshold_alarm2'] = df.apply(lambda row: categorise(row), axis = 1)
    
    conditions  = [ (df['consec_null'] == 0) & (df['consec_null_shift'].isnull()),
               (df['consec_null'] == 0) & (df['consec_null_shift'].notnull()),
            (df['consec_null'] != 0) & (df['consec_null_shift'].notnull())]
    choices     = [ 0, df['consec_null_shift'] + 1, df['consec_null_shift']]
    
    df["freq_judge"] = np.select(conditions, choices, default=np.nan)
    df['alarm2'] = np.where(df['freq_judge'] >= df['threshold_alarm2'], 1, 0)

# abnormal price
def alarm_3(df):
    group = df.groupby('entry_id_')["price_avg"]
    df['alarm3'] = group.transform(exceed_3sigma)

# constant price
def alarm_4(df):
    df['rank_'] = df.groupby(by = 'entry_id_')['date'].rank("dense", ascending = True)
    df['price_avg_shift'] = np.where(df['rank_'] == 1, np.nan, df['price_avg'].shift(1))
    df.drop(['rank_'], axis = 1, inplace = True)
    df['consec_count_same'] = (df['price_avg'] == df['price_avg_shift']).groupby(
        (df['price_avg'] != df['price_avg_shift']).cumsum()
    ).cumsum()
    threshold_alarm4 = 10
    df['alarm4'] = np.where(df['consec_count_same'] >= threshold_alarm4, 1, 0)
    
# change in number of entries in source 
def alarm_5(df):
    df_source = df.sort_values(by = ['date', 'source_id', 'entry_id_']).copy()
    entry_countBysource = pd.DataFrame(empty_df.groupby(
        by = ['date', 'source_id'])['entry_id_'].count()
                                      ).reset_index(level = (0,1))
    entry_countBysource = entry_countBysource.sort_values(by = ['source_id', 'date'])
    entry_countBysource['diff'] = entry_countBysource['entry_id_'] - entry_countBysource['entry_id_'].shift(1)
    entry_countBysource['Rank'] = entry_countBysource.groupby(by = 'source_id')['date'].rank("dense", ascending = True)
    entry_countBysource['diff_'] = np.where(entry_countBysource['Rank'] == 1, np.nan, entry_countBysource['diff'])
    entry_countBysource.drop(['Rank', 'diff'], axis = 1, inplace = True)

    threshold_alarm5 = 10
    result = entry_countBysource.loc[entry_countBysource['diff_'].abs() > threshold_alarm5]
    result_dict = result.to_dict('records')
    source_list = []
    date_list = []
    for result in result_dict:
        source_list.append(result['source_id'])
        date_list.append(result['date'])

    df['alarm5'] = np.where((df['source_id'].isin(source_list)) & (df['date'].isin(date_list)),
                           1, 0)

In [15]:
alarm_5(df)
alarm_4(df)
alarm_3(df)
alarm_2(df)
alarm_1(df)

freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40857878 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40858600 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40858670 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40859109 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40860060 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40860394 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40860587 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40861087 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40861531 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40862095 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40862096 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40862556 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40862858 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40863559 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40863754 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40863763 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40865627 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40867073 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 40867419 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry

freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 58933753 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 58944479 1.0
freq 최빈값이 [0, 1] 이외에도 있는 경우: entry, freq, 0 next 80961831 [0. 9.] 9.0
freq 최빈값이 [0, 3, 10...]일 경우, 0 next 9.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 80981371 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 80986712 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 80998534 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 81024533 1.0
freq 최빈값이 [0, 1] 이외에도 있는 경우: entry, freq, 0 next 81059630 [ 0. 13.] 13.0
freq 최빈값이 [0, 3, 10...]일 경우, 0 next 13.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 81101844 1.0
freq 최빈값이 [0, 1] 이외에도 있는 경우: entry, freq, 0 next 81177140 [0. 4.] 4.0
freq 최빈값이 [0, 3, 10...]일 경우, 0 next 4.0
freq 최빈값이 [0, 1] 이외에도 있는 경우: entry, freq, 0 next 81215662 [ 0. 14.] 14.0
freq 최빈값이 [0, 3, 10...]일 경우, 0 next 14.0
freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt =  81243393 0.0 23
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 81684101 1.0
freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt =  81779056 0.0 23
freq 최빈값이 단일값이며 

freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt =  104562265 0.0 14
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 104562287 1.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 104577352 1.0
freq 최빈값이 [0, 1] 이외에도 있는 경우: entry, freq, 0 next 104597087 [ 0.  3. 35.] 3.0
freq 최빈값이 [0, 3, 10...]일 경우, 0 next 3.0
freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt =  104630595 0.0 21
freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt =  104735346 0.0 10
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 104816189 1.0
freq 최빈값이 [0, 1]일 경우: entry, freq = 104943901 [0. 1.]
freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt =  105035479 0.0 17
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 105048612 1.0
freq 최빈값이 [0, 1] 이외에도 있는 경우: entry, freq, 0 next 105164076 [0. 1. 6.] 1.0
freq 최빈값이 [0, 1, 3, 10...]일 경우, 0 next 6.0
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 105167518 1.0
freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt =  105187962 0.0 10
freq 최빈값이 단일값이며 0이 아닐 경우: entry, freq 105192352 4.0
freq 최빈값이 단일값이며 0일 경우: entry, freq, timedelt =  105226177 0.0 10
freq 최빈값이 

In [25]:
df_latest = df.loc[(df['date'] >= '2022-03-01') & (df['date'] < '2022-05-31')].copy()

In [28]:
print(f'df_raw -> random sampling {len(sample_entry)} entries')
print('total number of entries: ', len(df_latest['entry_id_'].unique()))
print('alarm_5: change in # of entries in each source', len(df_latest[df_latest['alarm5'] == 1]))
print('alarm_4: constant price', len(df_latest[df_latest['alarm4'] == 1]))
print('alarm_3: abnormal price range', len(df_latest[df_latest['alarm3'] == 1]))
print('alarm_2: exceed price report frequency - # of entries ', len((df_latest.loc[df_latest['alarm2'] == 1]['entry_id_']).unique()))
print('alarm_1: price change rate (200%) ', len(df_latest[df_latest['alarm1'] == 1]))

print('''
## Point 1
alarm 5 threshold = 10: source 별로 묶여 있는 entry 개수가 너무 다르기 때문에, threshold를 entry마다 달리 설정할 필요성이 있어 보임. 기준을 별도로 정해야 할 필요성이 있음.
alarm 4 threshold = 10: 이는 기준을 비교적 정성적으로 정해도 될 것으로 판단 됨.
alarm 3 threshold = confidence level (99): 기준을 별도로 정해야 할 필요성이 있음.
alarm 2 threshold = 3: 이는 기준을 비교적 정성적으로 정해도 될 것으로 판단 됨.
alarm 1 threshold = 200%: 이는 기준을 비교적 정성적으로 정해도 될 것으로 판단 됨.
''')

df_raw -> random sampling 1000 entries
total number of entries:  998
alarm_5: change in # of entries in each source 116
alarm_4: constant price 61
alarm_3: abnormal price range 414
alarm_2: exceed price report frequency - # of entries  931
alarm_1: price change rate (200%)  9

## Point 1
alarm 5 threshold = 10: source 별로 묶여 있는 entry 개수가 너무 다르기 때문에, threshold를 entry마다 달리 설정할 필요성이 있어 보임. 기준을 별도로 정해야 할 필요성이 있음.
alarm 4 threshold = 10: 이는 기준을 비교적 정성적으로 정해도 될 것으로 판단 됨.
alarm 3 threshold = confidence level (99): 기준을 별도로 정해야 할 필요성이 있음.
alarm 2 threshold = 3: 이는 기준을 비교적 정성적으로 정해도 될 것으로 판단 됨.
alarm 1 threshold = 200%: 이는 기준을 비교적 정성적으로 정해도 될 것으로 판단 됨.



In [17]:
print(len(one_freq_0_lst1),
len(one_freq_0_lst2),
len(one_freq_not0_lst3),
len(multi_freq_lst1),
len(multi_freq_lst2),
len(multi_freq_lst3),
len(multi_freq_lst4))

14 142 703 5 40 25 69


In [18]:
alarm2_lst = df.loc[df['alarm2'] == 1]['entry_id_'].unique()

In [19]:
# mode final
df.groupby('entry_id_')['freq'].agg(pd.Series.mode).to_csv('final.csv')
# alarm2 sample
df.loc[(df['entry_id_'].isin(alarm2_lst))].to_csv('alarm2_sample.csv')
# individual sample
df.loc[(df['entry_id_'] == 123551904)].to_csv('sample_individual.csv')

In [ ]:
# df.drop(['entry_id', 'consec_null_shift', 'price_avg_shift', 'price_avg_chg_'])